### How to get ARLIE data
In the following, you can find a beginner-friendly guide on how to retrieve ARLIE data.

[The Copernicus Aggregated River and Lake Ice Extent (ARLIE) product](https://land.copernicus.eu/en/products/water-bodies/high-resolution-aggregated-river-and-lake-ice-extent) is a spatially aggregated information on surface water conditions of rivers and lakes. ARLIE information is stored in a geodatabase, enriched every day from the River and Lake Ice extent products (RLIE S1, RLIE S2 and RLIE S1+S2) for the entire EEA38 and the United Kingdom. It provides percent coverage of snow-covered or snow-free ice on lakes and on 10 km river sections described by the EU-HYDRO river and lake network database.<br>
<br>
The ARLIE products are stored in a PostGIS persistent geodatabase. They can be retrieved by using a specific REST API. Users can query ice sheet summary information of river segments and lakes (ARLIE statistics) together with the geometry and caracteristics of the features on which these statistics were estimated. All geometry features are delivered in the ETRS89 LAEA (EPSG:3035) coordinate reference system.<br>
<br>
ARLIE is one of the products of the pan-European High-Resolution Snow & Ice service (HR-S&I), which are provided at high spatial resolution (20 m x 20 m and 60 m x 60 m) from the Sentinel-2 and Sentinel-1 constellations data from September 1, 2016 onwards.<br>
<br>
This notebook contains the following steps:<br>
1) Initial setup
2) Pick your AOI
3) Import your geometry and display it
4) Transform to WKT format
5) Request ARLIE data
6) Display ARLIE data
7) Display EU Hydro data

It is important to note that data requests must adhere to the Well-Known Text (WKT) format.<br>
<br>
This notebook is sligthly modified version of [how-to-get-started notebook](https://github.com/eea/clms-hrsi-arlie-use-case/blob/main/how-to-get-started.ipynb) developed by EEA.


#### Step 1: Initial setup
1. First of all, pick your preferred platform for running Jupyter Notebooks<br>
*(For instance, you can opt for the [WekEO platform](https://help.wekeo.eu/en/articles/6337538-what-is-the-wekeo-jupyterhub))*<br>
2. Next, copy this GitHub repository or [the original one](https://github.com/eea/clms-hrsi-arlie-use-case.git) and paste all the included notebooks onto your chosen platform<br>
3. Make sure that you have the [clms_hrsi_arlie_downloader.py](clms_hrsi_arlie_downloader.py) located next to this notebook<br>
*You can see an example of the folder structure below*<br>
<img src="images/folders1.png"/><br>


4. Make sure you have installed all the necessary Python libraries. Additional to some standard Python libraries you will need: 
    - [Folium](https://pypi.org/project/folium/): A library that simplifies the creation of interactive maps using Python and Leaflet.js, allowing for easy visualization of geospatial data., 
    - [Geopandas](https://pypi.org/project/geopandas/): An extension of pandas designed to work with geospatial data, enabling operations like spatial joins and geometric manipulations using familiar DataFrame structures,
    - [Shapely](https://pypi.org/project/shapely/): A library for manipulating and analyzing planar geometric objects, providing tools for creating and working with points, lines, and polygons in a geometric context..<br>
<br>

_Other python libraries that could be quite be handy_:
- *[pandas](https://pandas.pydata.org/)*: A powerful library for data manipulation and analysis, ideal for handling structured data through DataFrames.
  
- *[matplotlib](https://matplotlib.org/)*: A versatile library for creating a wide range of static, animated, and interactive visualizations in Python, focusing on 2D plotting.

- *[pyplot](https://matplotlib.org/stable/api/pyplot_summary.html)*: A module within `matplotlib` that provides a MATLAB-like interface for creating plots easily and interactively.

- *[contextily](https://contextily.readthedocs.io/en/stable/)*: A library used to add basemaps (like satellite imagery or street maps) to geospatial visualizations, especially with `GeoPandas`.


In case you miss any libraries throughout this tutorial, simply install them by running `%pip install {package name}`
See example below:

In [ ]:
# press shift+enter to execute a cell

%pip install folium
%pip install Geopandas
%pip install Shapely

# to uncomment a line, select the line and press ctrl+/ or delete the # and space
# %pip install pandas
# %pip install matplotlib
# %pip install pyplot
# %pip install contextily

#### Step 2: Pick your AOI
To access ARLIE data for your specific AOI, you have two options. You can either upload a predefined geometry (e.g., a shapefile or GeoJSON) or you can draw a custom geometry on a map following the steps below. 

If you have a [predetermined geometry, jump directly to step 4](#step-4-transform-to-wkt-format). 

In case you [have a file in WKT format, jump to step 5](#step-5-request-arlie-data).

The geometry is used to request all lake and river polygons from the EU Hydro database within your AOI, which, in turn, is used to retrieve ARLIE data from the PostGIS database.


In [18]:
# Import some general libraries
import os
from pathlib import Path

#### Draw a custom geometry using Folium

When you execute the cell below, an interactive map will appear. You can use this map to draw your Area of Interest (AOI). Follow these steps:

1. Navigate the Map: Use the toolbar on the left to zoom in, zoom out, and move around the map.
2. Draw Your AOI: Click the drawing tool (polygon) in the toolbar and draw your desired AOI on the map.
3. Edit Geometry: If needed, use the toolbar to edit or refine the drawn geometry.
4. Export the Geometry: Once you're satisfied with your AOI, click the small 'Export' button on the right side of the toolbar. A window will appear prompting you to save the file.
5.  Save as GeoJSON: Save the file in GeoJSON format. You can save it either in the same folder as this notebook or anywhere on your computer.
6. Upload the GeoJSON: If you saved the file elsewhere, make sure to upload it into the same folder where this notebook is located to ensure the notebook can access it.
<br>
<br>


<img src="images/folium1.png" alt="Drawing" style="width: 400px;"/>

🤫 Note, the geometry should have an appropriate size and complexety, so try with a smaller area if you experience issues later in the workflow.

In [35]:
# Import relevant libraries
import folium
from folium.plugins import Draw

# Initialise an interactive map
m = folium.Map(location=[64.5, -18], zoom_start=7)

#  Add a function to draw and export your polygon
draw = Draw(export=True, filename='aoi.geojson', position='topleft')
draw.add_to(m)

# Display the map
m

#### Step 3: Import your geometry and display it

In [7]:
# Import relevant libraries
import folium
import geopandas as gpd
import os

# Create the 'data' folder if it doesn't exist and set the path to your AOI inside the 'data' folder
os.makedirs('data', exist_ok=True)
aoi_path = os.path.join('data', 'aoi.geojson')

# Read the AOI as a geodataframe
gdf = gpd.read_file(aoi_path)

# Retrieve the center points of the AOI to display it later
aoiX = gdf["geometry"].centroid.x
aoiY = gdf["geometry"].centroid.y

# Add your AOI to an interactive map and display it
m = folium.Map(
    location=[aoiY[0],aoiX[0]],
    tiles='OpenStreetMap',
    zoom_start=7,
    min_zoom=1,
    max_zoom=15,
    control_scale=True)

folium.GeoJson(aoi_path, name="AOI", style_function=lambda x: {
    'fillOpacity': 0.1,
    'color': 'Crimson'
    }).add_to(m)

# Display the map
m

C:\Users\michaela.hrabalikova\AppData\Local\Temp\ipykernel_45296\1698464263.py:14: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  aoiX = gdf["geometry"].centroid.x
C:\Users\michaela.hrabalikova\AppData\Local\Temp\ipykernel_45296\1698464263.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  aoiY = gdf["geometry"].centroid.y


#### Step 4: Transform to WKT format
Transform your AOI from the Geojson to WKT format using Shapely. In case you uploaded a shapefile, simply use this as input.

In [21]:
# Import relevant libraries
import json
from shapely.geometry import shape

# Set the path to your AOI
aoi = "aoi.geojson"

# Use Shapely to transform to WKT format and save it
with open(Path(aoi)) as fin, open(Path(aoi).with_suffix(".wkt"), "w") as fout:
    features = json.load(fin)["features"]
    for feature in features:
        geo = shape(feature["geometry"])
        wkt = geo.wkt
        fout.write(wkt + "\n")

In [22]:
# A WKT (Well-Known Text) is a scheme for writing a geometry into a standard text string, as you can see below
wkt

'POLYGON ((7.106781 60.426055, 7.106781 60.657032, 7.68631 60.657032, 7.68631 60.426055, 7.106781 60.426055))'

#### Step 5: Request ARLIE data
The retrieval of the ARLIE data can take a while, depending on the size and complexety of your AOI. In the end, you should see a message informing on where the output has been saved and how many ARLIE records have been found.

In [23]:
# Import the API to download ARLIE data
from clms_hrsi_arlie_downloader import download_arlie_products

# Set the following variables to retrieve ARLIE data:
geometryWkt = wkt               # The WKT you have generated above, or simply copy+paste a geometry directly (eg. 'POLYGON ((9.667968999999999 52.48278, 10.722656 51.289406, 6.855469 49.496675, 10.283203 47.457809, 15.205078 48.922499, 14.238281 51.344339, 9.667968999999999 52.48278))')
outputDir = "./output"          # Your preferred output folder
startDate = '2017-01-01'        # Start date for ARLIE data
completionDate = '2023-01-01'   # End date for ARLIE data
cloudCoverageMax = 100          # Maximum cloud cover to consider
requestGeometries = True        # True returns polygons from EU Hydro, False only returns the ARLIE data
returnMode = 'csv_and_variable' # choose between "csv", "csv_and_variable", "variable"

geom, arlie = download_arlie_products(returnMode, outputDir=outputDir, startDate=startDate, completionDate=completionDate, geometryWkt=geometryWkt, cloudCoverageMax=cloudCoverageMax, requestGeometries=requestGeometries)

Executing request for geometries: https://cryo.land.copernicus.eu/arlie/get_geometries?geometrywkt=POLYGON+((7.106781+60.426055%2C+7.106781+60.657032%2C+7.68631+60.657032%2C+7.68631+60.426055%2C+7.106781+60.426055))&getonlyids=True
Executing request for geometries: https://cryo.land.copernicus.eu/arlie/get_geometries?geometrywkt=POLYGON+((7.106781+60.426055%2C+7.106781+60.657032%2C+7.68631+60.657032%2C+7.68631+60.426055%2C+7.106781+60.426055))
Writing geometries in /home/jovyan/Arlie_repoEL/output/geometries.csv
Executing request for ARLIE: https://cryo.land.copernicus.eu/arlie/get_arlie?geometrywkt=POLYGON+((7.106781+60.426055%2C+7.106781+60.657032%2C+7.68631+60.657032%2C+7.68631+60.426055%2C+7.106781+60.426055))&cloudcoveragemax=100&startdate=2017-01-01&completiondate=2023-01-01
Done
Writing ARLIE in /home/jovyan/Arlie_repoEL/output/arlie.csv
Found 441725 ARLIE products.
Writing metadata link into /home/jovyan/Arlie_repoEL/output/ARLIE_MTD.xml
End.


#### Step 6: Display ARLIE data
We now retrieved the ARLIE data in a csv format and saved it to the folder <code style="background:yellow;color:black">output</code>.
The data should look similar to this when opening as a list:
> [[1, 46493, '2019-01-01T17:16:05', 27, 56, 0, 0, 17 ,1 ,'Sentinel-1 Sentinel-2'],
 [2, 46493, '2019-01-02T17:07:26', 62, 23, 0, 0, 15, 1, 'Sentinel-1 Sentinel-2']], [3, 46493, '2019-01-03T16:59:52', 51, 35, 0, 0, ...

Pass the statistics into a Pandas DataFrame to easier explore and organise the data by running the cell below.

In [24]:
# Import relevant libraries
import pandas as pd
from datetime import datetime

# Parse the orignial ARLIE date format to something more readable
dateparse = lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S')

# Read the csv file into a dataframe and display it
arlie = pd.read_csv(os.path.join(outputDir, 'arlie.csv'), parse_dates=['datetime'], delimiter=";")
arlie.to_csv(os.path.join(outputDir, 'arlie.csv'))
arlie

,id,river_km_id,datetime,water_perc,ice_perc,other_perc,cloud_perc,nd_perc,qc,source
0,1,378906,2017-01-01 05:48:00,67,0,0,0,33,2,Sentinel-1
1,2,378888,2017-01-01 05:48:00,85,0,0,0,15,2,Sentinel-1
2,3,378927,2017-01-01 05:48:00,91,0,0,0,9,2,Sentinel-1
3,4,378991,2017-01-01 05:48:00,71,8,0,0,21,2,Sentinel-1
4,5,378996,2017-01-01 05:48:00,91,0,0,0,9,2,Sentinel-1
...,...,...,...,...,...,...,...,...,...,...
441720,441721,379971,2022-02-28 17:03:54,82,14,0,0,4,2,Sentinel-1 Sentinel-2
441721,441722,379972,2022-02-28 17:03:54,96,0,0,0,4,2,Sentinel-1 Sentinel-2
441722,441723,379974,2022-02-28 17:03:54,86,9,0,0,5,2,Sentinel-1 Sentinel-2
441723,441724,379887,2022-02-28 17:03:54,68,17,0,0,15,2,Sentinel-1 Sentinel-2


The output should now look similar to the table above, with the following columns:<p>

>**id** = unique identifcation number for each ARLIE record <br>
**river_km_id** = identification number for the lake or river section polygon from the EU Hydro database<br>
**datetime** = date and time of ARLIE record<br>
**water_perc** = amount of water per waterbody in %<br>
**ice_perc** = amount of ice or snow cover per waterbody in %<br>
**other_perc** = amount of other features per waterbody in %<br>
**cloud_perc** = cloud cover per waterbody in %<br>
**nd_perc** = no data pixels per waterbody %<br>
**source** = satellite data used to derive ARLIE record<br>
**qc** = quality control value with the following quality levels:
>> high quality: 0.5 > ARLIE_confidence ≥ 0<br>
>> medium quality: 1.5 > ARLIE_confidence ≥ 0.5<br>
>> low quality: 2.5 > ARLIE_confidence ≥ 1.5<br>
>> minimal quality: 3.0 ≥ ARLIE_confidence ≥ 2.5<br>
    
To organise the data, you can sort it by date and filter it for the highest quality level. Do do this, run the cell below.

In [25]:
# Sort records by datetime
arlie = arlie.sort_values(by="datetime", ascending=True)

# Only get records with high confidence (i.e. QC flag = 0)
arlie = arlie.where(arlie.qc == 0).dropna()
display(arlie)

,id,river_km_id,datetime,water_perc,ice_perc,other_perc,cloud_perc,nd_perc,qc,source
5921,5922.0,379724.0,2017-01-26 16:54:35,0.0,41.0,0.0,51.0,8.0,0.0,Sentinel-1 Sentinel-2
6094,6095.0,379449.0,2017-01-26 16:54:35,0.0,0.0,0.0,83.0,17.0,0.0,Sentinel-1 Sentinel-2
6091,6092.0,379500.0,2017-01-26 16:54:35,0.0,9.0,0.0,80.0,11.0,0.0,Sentinel-1 Sentinel-2
5813,5814.0,379076.0,2017-01-26 16:54:35,0.0,0.0,0.0,88.0,12.0,0.0,Sentinel-1 Sentinel-2
5808,5809.0,379040.0,2017-01-26 16:54:35,0.0,25.0,0.0,56.0,19.0,0.0,Sentinel-1 Sentinel-2
...,...,...,...,...,...,...,...,...,...,...
406747,406748.0,378965.0,2022-12-31 10:54:38,0.0,0.0,0.0,82.0,18.0,0.0,Sentinel-2
406746,406747.0,378960.0,2022-12-31 10:54:38,0.0,0.0,0.0,93.0,7.0,0.0,Sentinel-2
406745,406746.0,378950.0,2022-12-31 10:54:38,0.0,0.0,0.0,88.0,12.0,0.0,Sentinel-2
406743,406744.0,378941.0,2022-12-31 10:54:38,0.0,0.0,0.0,82.0,18.0,0.0,Sentinel-2


### Step 7: Display EU Hydro data

Along with the ARLIE statistics, we retrieved EU Hydro data in csv format. Those contain vector polygon data with information about the lake/river ID, lake/river basin name, EU Hydro ID, object name, area, and river kilometre from the mouth (only for rivers which are split into 10 km long sections). Note, that we only recieve the individual EU Hydro ID numbers and no river/lake names. Read more about the full EU Hydro dataset and retrieve the river/lake names [here](#https://www.eea.europa.eu/en/datahub/datahubitem-view/2e782ca5-c7b2-4b48-8928-03031b642176).

By passing the csv file into a Pandas GeoDataFrame we can display the geometries on a folium map. Run the cells below to see how this works.

In [26]:
from shapely.wkt import loads

# Read the geometries from csv file
geometries = pd.read_csv(os.path.join(outputDir, 'geometries.csv'), delimiter=";")

# Convert the WKT geometry strings to Shapely geometry objects
geometries['geometry'] = geometries['geometry'].apply(loads)

# Create a GeoDataFrame. As the ARLIE data comes in the LAEA European projection, we use the crs EPSG:3035
EUhydro_gdf = gpd.GeoDataFrame(geometries, geometry='geometry', crs='EPSG:3035')
EUhydro_gdf.head()

,id,geometry,basin_name,eu_hydro_id,object_nam,area,river_km
0,379338,"MULTIPOLYGON (((4164983.064 4162193.428, 41649...",Vorma,IW40032787,UNK,19931.63,NaN
1,379412,"MULTIPOLYGON (((4163971.649 4163710.841, 41639...",Vorma,IW40032861,UNK,20109.02,NaN
2,379666,"MULTIPOLYGON (((4163542.631 4168932.734, 41635...",Vorma,IW40033115,UNK,20765.77,NaN
3,379927,"MULTIPOLYGON (((4168470.905 4174170.158, 41684...",Vorma,IW40033376,UNK,28225.79,NaN
4,379866,"MULTIPOLYGON (((4163919.531 4172605.933, 41638...",Vorma,IW40033315,UNK,44720.36,NaN


In [27]:
# Set the path to your AOI in geojson format and read it as GeoDataFrame
aoi = "aoi.geojson"
aoi_gdf = gpd.read_file(aoi)

# Retrieve the center points of the AOI to display it later
aoiX = aoi_gdf["geometry"].centroid.x
aoiY = aoi_gdf["geometry"].centroid.y

# Create an empty map
m = folium.Map(
    location=[aoiY[0],aoiX[0]],
    tiles='OpenStreetMap',
    zoom_start=7,
    min_zoom=1,
    max_zoom=15,
    control_scale=True
)

# Add the EU Hydro data and your AOI to an interactive map and display it
folium.GeoJson(EUhydro_gdf, name="EU Hydro rivers & Lakes", style_function=lambda x: {'color': 'RoyalBlue'}).add_to(m)
folium.GeoJson(aoi, name="AOI", style_function=lambda x: {'fillOpacity': 0.1,'color': 'Crimson'}).add_to(m)

folium.LayerControl().add_to(m)

m

/tmp/ipykernel_647/2345243736.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  aoiX = aoi_gdf["geometry"].centroid.x
/tmp/ipykernel_647/2345243736.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  aoiY = aoi_gdf["geometry"].centroid.y


Awesome! 🥳 You now requested ARLIE data and the assoiciated EU Hydro polygons for your AOI! Use it as you like or continue with the next notebook if you a curious on how to analyse and manipulate the data further.